In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys

In [2]:
class Neuron:
    """Neuron in neural network
    
    __init__(num_in,bias)
        num_in = number of features to input to neuron (creates the number of weights on the neuron)
        bias = True if a bias term should be included for w*x+b 
               False if no bias term should be included for w*x+b
               default=True
           

    forward(x)
        x = input to the neuron for w*x + b
    
    
    backward(gradient_l)
        gradient_l = the upstream gradient for backpropogation
        gradient_x = the local gradient of input x
        gradient_w = the local gradient of the neuron weights w
        
        return the local gradients * the upstream gradient
    """
    
    def __init__(self,num_in,bias=True):
        self.gradient_x = None
        self.gradient_w = None
        
        if bias:
            num_in = num_in + 1
            
        self.weights = np.random.uniform(-1,1,size=(num_in,1))
        self.x = None
        self.bias=bias
    
    def forward(self,x):
        if self.bias == True:
            x = np.concatenate((x,[[1]]),axis=1)
            
        self.x = x
        return np.matmul(x,self.weights)
                                
    def backward(self,gradient_l=1):
        if self.x is not None:
            self.gradient_x = self.weights.transpose()*gradient_l
            self.gradient_w = self.x.transpose()*gradient_l
            return self.gradient_x,self.gradient_w
        else:
            return None          

In [3]:
class ReLu:
    """ReLu Activation function
    
    forward(x)
    x = the input to sigmoid function max(0,x)
        
    backward(gradient_l)
        gradient_l = the upstream gradient for backpropogation
        self.gradient = the local gradient of the Sigmoid activation function
        
        return the local gradients * the upstream gradient
    """
    def __init__(self):
        self.gradient = None
        self.x = None
    
    def forward(self,x):
        self.x = x
        if x > 0:
            return x
        else:
            return 0
    
    def backward(self,gradient_l=1):
        if self.x is not None:
            if self.x > 0:
                self.gradient = gradient_l
                return gradient_l
            else:
                self.gradient = sys.float_info.epsilon
                return np.array([[sys.float_info.epsilon]])   
        else:
            return None

In [4]:
class Sigmoid:
    """Sigmoid Activation function
    
    forward(x)
        x = the input to sigmoid function f(x) = 1 / (1+exp(-x))
        
    backward(gradient_l)
        gradient_l = the upstream gradient for backpropogation
        self.gradient = the local gradient of the Sigmoid activation function
        
        return the local gradients * the upstream gradient
    """
    def __init__(self):
        self.gradient=None
        self.x = None
        
    def forward(self,x):
        self.x = x
        return 1/(1+np.exp(-x))
    
    def backward(self,gradient_l=1):
        if self.x is not None:
            self.gradient = 1/(1+np.exp(-self.x))*(1-1/(1+np.exp(-self.x)))*gradient_l
            return self.gradient
        else:
            return None

In [5]:
class BCE:
    """Binary Cross Entropy function
    
    forward(y_hat,y)
        y_hat = the predicted classification probability 0-1
        y = the training example actual label 0 or 1
        binary cross entropy = -(y*ln(y_hat) + (1-y)*ln(1-y_hat))
        
    backward(gradient_l)
        gradient_l = the upstream gradient for backpropogation
        self.gradient = the local gradient of the ReLu activation function
        
        return the local gradients * the upstream gradient
    """
    def __init__(self):
        self.gradient = None
        self.y = None
        self.y_hat = None
    
    def forward(self,y_hat,y):
        self.y_hat = y_hat
        self.y = y
        return -(y*np.log(max(sys.float_info.epsilon,y_hat-sys.float_info.epsilon)) + \
                 (1-y)*np.log(1-max(sys.float_info.epsilon,y_hat-sys.float_info.epsilon)))
    
    def backward(self,gradient_l=1):
        if self.y is not None and self.y_hat is not None:
            self.gradient = -self.y*1/(max(sys.float_info.epsilon,self.y_hat)) + \
                              1/max(sys.float_info.epsilon,1-self.y_hat)*(1-self.y)
            return self.gradient
        else:
            return None
        
    def __str__(self):
        return "loss"

In [6]:
class neural_layer:
    """Neural Network layer
    
    __init__(input_,output_,bias=True)
        input_ = the number of features in the training example
        output_ = the number of neurons to use in the neural network layer
        bias = True if a bias term should be included for w*x+b 
               False if no bias term should be included for w*x+b
               default=True
    
    forward(x):
        x = input to the neuron for W*x + B
        
    backward(gradient_l):
        gradient_l = the upstream gradient for backpropogation
        gradients has gradient_x and gradient_w for each neuron in the neural network layer
        
        return the local gradients * the upstream gradient
    """
    def __init__(self,input_= 1,output_= 1,bias=True):
        self.output_size = output_
        self.input_size = input_
        self.neurons = [Neuron(input_,bias) for _ in range(output_)]
        
    def forward(self,x):
        outputs = [neuron.forward(x) for neuron in self.neurons]
        return np.concatenate(outputs,1)
    
    def backward(self,gradient_l=None):
        if gradient_l is None:
            return None
        
        gradients = []
        gradient_l = gradient_l.squeeze(0)       
        
        for i,neuron in enumerate(self.neurons):
            gradients.append(neuron.backward(gradient_l[i]))
            
        return gradients
    
    def __str__(self):
        return "neuron"

In [7]:
class action_layer:
    """Activation layer
    
    __init__(output_,a_type="ReLu")
        output_ = the number of activation units to use in the neural network layer
        a_type = the type of activation unit you want to use: "ReLu" or "Sigmoid"
    
    forward(x):
        x = input to the activation layer
        return the activation evaluation
        
    backward(gradient_l):
        gradient_l = the upstream gradient for backpropogation
        gradients has the local gradients for each activation unit
        
        return the local gradients * the upstream gradient
    """
    def __init__(self,output_= 1,a_type="ReLu"):
        
        self.output_size = output_
        
        if a_type == "ReLu":
            self.activations = [ReLu() for _ in range(output_)]
        if a_type == "Sigmoid":
            self.activations = [Sigmoid() for _ in range(output_)]
        
    def forward(self,x):
        output = []
        x = x.squeeze(0)
        for i,activation in enumerate(self.activations):
            output.append(activation.forward(x[i]))
        return np.expand_dims(output,0)
    
    def backward(self,gradient_l=None):
        if gradient_l is None:
            return None
        
        gradients = []
        gradient_l = gradient_l.squeeze(1)     
        
        for i,activation in enumerate(self.activations):
            gradients.append(activation.backward(gradient_l[i]))
            
        return np.expand_dims(gradients,0)
    
    def __str__(self):
        return "activation"

In [8]:
class neural_network:
    """Neural Network
    
    __init__(model=[])
        model = a list [] of all the neural network layer objects and activation layer objects. The forward and backward execution
                is sequential in regards to the order of the list passed in to model
    
    forward(x)
        x = the input training example to the neural network which will pass through all the layers of neural network and activation
            layers to classify or regress
    
    backward(gradient_l)
        gradient_l = the upstream gradient from the loss function to the neural network

        return all the gradients calculated throughout all the layers  
        
    model_weights()
        return a list with all the model weights for each neural network layer with neurons
    """
    def __init__(self,model=[]):
        self.model = model
        self.gradients = []
        
    def forward(self,x):
        for obj in self.model:
            x = obj.forward(x)    
        return x
    
    def backward(self,gradient_l=None):
        self.gradients = []
        for obj in self.model[::-1]:
            if str(obj) == "activation":
                tup = ("activation",obj.backward(gradient_l))
                self.gradients.append(tup)
                gradient_l = tup[1]
            
            elif str(obj) == "neuron":
                tup = ("neuron",obj.backward(gradient_l))
                self.gradients.append(tup)
                
                if gradient_l is None:
                    gradient_l = None
                    continue
                
                gradient_l = 0
                for grad in tup[1]:
                    gradient_l += grad[1]  

        self.gradients = self.gradients[::-1]
    
    def model_weights(self):
        weights = []
        layer_num = 0
        for obj in self.model:
            if str(obj) == 'neuron':
                layer_weights = []
                for neuron in obj.neurons:
                    layer_weights.append(neuron.weights)
                weights.append(('layer'+str(layer_num),layer_weights))
                layer_num += 1
        return weights
        

In [42]:
class optimizer:
    
    def __init__(self,model = None,alpha=0.05):
        self.model = model
        self.loss_fn = loss_fn
        self.alpha = alpha

    def step(self):
        if model is not None:
            for obj,grad in zip(self.model.model,nn_back):
                if grad[0] == 'neuron':
                    for n,grad_update in zip(obj.neurons,grad[1]):
                        n.weights = n.weights - self.alpha*grad_update[1]

# Create Dataset (use the breast cancer dataset)
* Download and use the breast cancer dataset with sklearn
* standard normalize the data

In [43]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()

In [44]:
X = data['data']
X = X - np.mean(X,0)
X = X / np.std(X,0)
Y = data['target']

# Attempt to do logistic regression proof of concept

In [45]:
neuron = Neuron(30)

In [46]:
activation = Sigmoid()

In [47]:
loss_fn = BCE()

In [48]:
alpha = 0.5

for i in range(10):
    mean_loss = []
    for x,y in zip(X,Y):
        x = np.expand_dims(x,0)

        # forward pass
        output = neuron.forward(x)
        a = activation.forward(output)
        
        loss = loss_fn.forward(a,y)
        mean_loss.append(loss)

        # backward pass
        loss_grad = loss_fn.backward()
        a_grad = activation.backward(loss_grad)
        _,weight_grad = neuron.backward(a_grad)

        # update model
        neuron.weights = neuron.weights - alpha*weight_grad
    print(i,np.mean(mean_loss))

0 [[0.17052478]]
1 [[0.16366847]]
2 [[0.11125718]]
3 [[0.16984805]]
4 [[0.13401785]]
5 [[0.10970913]]
6 [[0.12862981]]
7 [[0.12040971]]
8 [[0.17409925]]
9 [[0.11903616]]


In [49]:
y_predictions = []
for x,y in zip(X,Y):
    x = np.expand_dims(x,0)

    # forward pass
    output = neuron.forward(x)
    a = activation.forward(output)
    
    y_predictions.append(a)

In [50]:
y_pred = np.round(np.array(y_predictions).squeeze())

In [51]:
accuracy = np.mean(y_pred == Y)
print(accuracy)

0.9859402460456942


# Neural Network proof of concept

In [84]:
model = [neural_layer(30,10),action_layer(10,"Sigmoid"),neural_layer(10,5),action_layer(5,"Sigmoid"),neural_layer(5,1),action_layer(a_type='Sigmoid')]
nn = neural_network(model)
loss_fn = BCE()

alpha = 0.5

optimize = optimizer(nn,alpha)

In [86]:
for i in range(20):
    mean_loss = []
    for x,y in zip(X,Y):
        x = np.expand_dims(x,0)

        # forward pass
        output = nn.forward(x)
        loss = loss_fn.forward(output,y)
        mean_loss.append(loss)

        # backward pass
        loss_back = loss_fn.backward()
        nn_back = nn.backward(loss_back)

        # update weights based on gradient
        optimize.step()

    print(i,np.mean(mean_loss))

0 0.019578808213059233
1 0.018708816006499547
2 0.01787551369690303
3 0.017087242886214726
4 0.016346278416418565
5 0.01565111624231706
6 0.014998447481860504
7 0.0143844680921526
8 0.013805597103413197
9 0.013258795888104519
10 0.012741654192541987
11 0.01225235008189694
12 0.011789544849622649
13 0.011352247453429769
14 0.010939671053489348
15 0.010551099646602074
16 0.010185779764168209
17 0.009842847381286223
18 0.009521293169986261
19 0.009219961888920346


In [90]:
y_predictions = []
for x,y in zip(X,Y):
    x = np.expand_dims(x,0)

    # forward pass
    output = nn.forward(x)
    
    y_predictions.append(output)

In [91]:
y_pred = np.round(np.array(y_predictions).squeeze())

In [92]:
accuracy = np.mean(y_pred == Y)
print(accuracy)

0.9894551845342706
